# 월마트 판매

## #01. 준비작업

### [1] 패키지 참조

In [376]:
import sys
import os
work_path = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
sys.path.append(work_path)

from helper.regrassion import *
from helper.util import *
from helper.plot import *
from helper.analysis import *

### [2] 데이터 가져오기

주요 소매업체의 매출에 영향을 미칠 수 있는 요소 찾기

https://www.kaggle.com/datasets/mikhail1681/walmart-sales


| 변수명 | 설명 |
|:--:|--:|
| Store | Store number |
| Date | Sales week start date |
| Weekly_Sales | Store week sales |
| Holiday_Flag | Mark on the presence or absence of a holiday |
| Temperature | Air temperature in the region |
| Fuel_Price | Fuel cost in the region |
| CPI | Consumer price index(소비자 물가 지수) |
| Unemployment | Unemployment rate |

In [377]:
origin = my_read_data("./res/Walmart_sales.csv", info = False)
my_pretty_table(origin.head())

+----+---------+------------+----------------+----------------+---------------+--------------+---------+----------------+
|    |   Store | Date       |   Weekly_Sales |   Holiday_Flag |   Temperature |   Fuel_Price |     CPI |   Unemployment |
|----+---------+------------+----------------+----------------+---------------+--------------+---------+----------------|
|  0 |       1 | 05-02-2010 |    1.64369e+06 |              0 |         42.31 |        2.572 | 211.096 |          8.106 |
|  1 |       1 | 12-02-2010 |    1.64196e+06 |              1 |         38.51 |        2.548 | 211.242 |          8.106 |
|  2 |       1 | 19-02-2010 |    1.61197e+06 |              0 |         39.93 |        2.514 | 211.289 |          8.106 |
|  3 |       1 | 26-02-2010 |    1.40973e+06 |              0 |         46.63 |        2.561 |  211.32 |          8.106 |
|  4 |       1 | 05-03-2010 |    1.55481e+06 |              0 |          46.5 |        2.625 |  211.35 |          8.106 |
+----+---------+--------

In [378]:
df = origin.copy()



In [379]:
# 주차별 평균 / 중간값

df_date = df[['Date','Weekly_Sales']].groupby('Date').mean().sort_values('Weekly_Sales').reset_index()
map_date = {}
for i,v in enumerate(df_date['Date']):
    map_date[v] = i//29
df['Date_value'] = df['Date'].map(map_date)
map_date

{'28-01-2011': 0,
 '27-01-2012': 0,
 '31-12-2010': 0,
 '21-01-2011': 0,
 '14-01-2011': 0,
 '24-09-2010': 0,
 '13-01-2012': 0,
 '20-01-2012': 0,
 '30-09-2011': 0,
 '01-10-2010': 0,
 '23-09-2011': 0,
 '07-01-2011': 0,
 '25-03-2011': 0,
 '22-10-2010': 0,
 '17-09-2010': 0,
 '15-10-2010': 0,
 '01-04-2011': 0,
 '29-04-2011': 0,
 '29-10-2010': 0,
 '29-07-2011': 0,
 '30-04-2010': 0,
 '27-04-2012': 0,
 '28-09-2012': 0,
 '16-09-2011': 0,
 '26-02-2010': 0,
 '20-05-2011': 0,
 '27-07-2012': 0,
 '25-02-2011': 0,
 '26-03-2010': 0,
 '14-09-2012': 1,
 '21-09-2012': 1,
 '14-10-2011': 1,
 '11-03-2011': 1,
 '30-07-2010': 1,
 '23-04-2010': 1,
 '18-03-2011': 1,
 '23-07-2010': 1,
 '06-01-2012': 1,
 '15-04-2011': 1,
 '19-03-2010': 1,
 '23-03-2012': 1,
 '20-04-2012': 1,
 '08-10-2010': 1,
 '21-05-2010': 1,
 '19-10-2012': 1,
 '19-11-2010': 1,
 '16-04-2010': 1,
 '30-03-2012': 1,
 '22-07-2011': 1,
 '27-05-2011': 1,
 '14-05-2010': 1,
 '02-09-2011': 1,
 '13-05-2011': 1,
 '15-07-2011': 1,
 '26-10-2012': 1,
 '10-09-20

In [380]:
df_store = df[['Store','Weekly_Sales']].groupby('Store').mean().sort_values('Weekly_Sales').reset_index()
map_store = {}
for i,v in enumerate(df_store['Store']):
    map_store[v] = i//9
df['Store_value'] = df['Store'].map(map_store)
map_store


{33: 0,
 44: 0,
 5: 0,
 36: 0,
 38: 0,
 3: 0,
 30: 0,
 37: 0,
 16: 0,
 29: 1,
 9: 1,
 42: 1,
 7: 1,
 15: 1,
 43: 1,
 25: 1,
 21: 1,
 45: 1,
 17: 2,
 8: 2,
 35: 2,
 40: 2,
 34: 2,
 26: 2,
 12: 2,
 22: 2,
 18: 2,
 32: 3,
 41: 3,
 28: 3,
 11: 3,
 24: 3,
 23: 3,
 31: 3,
 19: 3,
 39: 3,
 1: 4,
 6: 4,
 27: 4,
 10: 4,
 2: 4,
 13: 4,
 14: 4,
 4: 4,
 20: 4}

In [381]:
df2 = df.drop(['Store','Date'],axis=True)
df2

,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value
0,1643690.90,0,42.31,2.572,211.096358,8.106,4,4
1,1641957.44,1,38.51,2.548,211.242170,8.106,4,4
2,1611968.17,0,39.93,2.514,211.289143,8.106,3,4
3,1409727.59,0,46.63,2.561,211.319643,8.106,0,4
4,1554806.68,0,46.50,2.625,211.350143,8.106,2,4
...,...,...,...,...,...,...,...,...
6430,713173.95,0,64.88,3.997,192.013558,8.684,0,1
6431,733455.07,0,64.89,3.985,192.170412,8.667,3,1
6432,734464.36,0,54.47,4.000,192.327265,8.667,2,1
6433,718125.53,0,56.47,3.969,192.330854,8.667,1,1


In [382]:
# my_pairplot(df2)

In [383]:
df2['Holiday_Flag'] = df2['Holiday_Flag'].astype('uint8') 
df2.dtypes

Weekly_Sales    float64
Holiday_Flag      uint8
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
Date_value        int64
Store_value       int64
dtype: object

In [384]:
temperature = []
for i in df2['Temperature']:
    temperature.append((i-32)*5/9)
df2['Temperature'] = temperature
df2.columns

Index(['Weekly_Sales', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI',
       'Unemployment', 'Date_value', 'Store_value'],
      dtype='object')

In [385]:
df2['Temperature2'] = df2['Temperature']**2
df2['Temperature3'] = df2['Temperature']**3
df2['Fuel_Price2'] = df2['Fuel_Price']**2
df2['CPI2'] = df2['CPI']**2
df2['Unemployment2'] = df2['Unemployment']**2


In [386]:
df3 = my_replace_outliner_to_nan(df2, 'Weekly_Sales').dropna()
df2.describe()

,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value,Temperature2,Temperature3,Fuel_Price2,CPI2,Unemployment2
count,6.435000e+03,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000,6435.000000
mean,1.046965e+06,0.069930,15.924324,3.358607,171.578394,7.999151,1.972028,2.000000,358.572563,8691.572376,11.490906,30987.855331,67.504814
std,5.643666e+05,0.255049,10.247185,0.459020,39.356712,1.875885,1.404120,1.414323,314.203222,10006.412494,3.068967,13617.643212,34.769820
min,2.099862e+05,0.000000,-18.922222,2.472000,126.064000,3.879000,0.000000,0.000000,0.000123,-6775.111011,6.110784,15892.132096,15.046641
25%,5.533501e+05,0.000000,8.588889,2.933000,131.735000,6.891000,1.000000,1.000000,77.880633,633.601008,8.602489,17354.110225,47.485881
50%,9.607460e+05,0.000000,17.038889,3.445000,182.616521,7.874000,2.000000,2.000000,290.513086,4946.793855,11.868025,33348.793560,61.999876
75%,1.420159e+06,0.000000,23.855556,3.735000,212.743293,8.622000,3.000000,3.000000,569.087562,13575.901417,13.950225,45259.708921,74.338884
max,3.818686e+06,1.000000,37.855556,4.468000,227.232807,14.313000,4.000000,4.000000,1433.043086,54248.642171,19.963024,51634.748486,204.861969


In [387]:
my_anova(df3[['Holiday_Flag','Weekly_Sales']],hue = 'Holiday_Flag',target = 'Weekly_Sales')

pingouin.anova
+----+--------------+-------------+------+-------------+---------+-----------+-------------+
|    | Source       |          SS |   DF |          MS |       F |     p-unc |         np2 |
|----+--------------+-------------+------+-------------+---------+-----------+-------------|
|  0 | Holiday_Flag | 1.22323e+12 |    1 | 1.22323e+12 | 4.11732 | 0.0424875 | 0.000643018 |
|  1 | Within       |  1.9011e+15 | 6399 | 2.97094e+11 |     nan |       nan |         nan |
+----+--------------+-------------+------+-------------+---------+-----------+-------------+

statsmodels.anova.anova_lm
+-----------------+------+-------------+-------------+---------+-----------+
|                 |   df |      sum_sq |     mean_sq |       F |    PR(>F) |
|-----------------+------+-------------+-------------+---------+-----------|
| C(Holiday_Flag) |    1 | 1.22323e+12 | 1.22323e+12 | 4.11732 | 0.0424875 |
| Residual        | 6399 |  1.9011e+15 | 2.97094e+11 |     nan |       nan |
+-------------

In [388]:
df4 = my_standard_scaler(df3, 'Weekly_Sales')
df4.head()

,Temperature,Fuel_Price,CPI,Unemployment,Date_value,Store_value,Temperature2,Temperature3,Fuel_Price2,CPI2,Unemployment2,Holiday_Flag,Weekly_Sales
0,-1.002520,-1.713516,1.002474,0.055203,1.456337,1.425277,-1.041902,-0.853426,-1.589018,0.995032,-0.053358,0,1643690.90
1,-1.208867,-1.765728,1.006179,0.055203,1.456337,1.425277,-1.104687,-0.867466,-1.629003,0.999554,-0.053358,1,1641957.44
2,-1.131758,-1.839696,1.007372,0.055203,0.742006,1.425277,-1.084545,-0.863652,-1.685008,1.001011,-0.053358,0,1611968.17
3,-0.767937,-1.737446,1.008147,0.055203,-1.400985,1.425277,-0.936067,-0.818575,-1.607391,1.001957,-0.053358,0,1409727.59
4,-0.774996,-1.598213,1.008922,0.055203,0.027676,1.425277,-0.939787,-0.819992,-1.499389,1.002904,-0.053358,0,1554806.68


In [389]:
yname = 'Weekly_Sales'
x_train, x_test, y_train, y_test = my_train_test_split(df4, yname, test_size=0.2)
my_linear_regrassion(x_train, y_train, x_test, y_test, use_plot=False, report=True)

[회귀식]
Weekly_Sales = 35815.123 * Temperature + 238894.568 * Fuel_Price + 251112.373 * CPI + -43582.101 * Unemployment + 68564.346 * Date_value + 511842.428 * Store_value + -33671.012 * Temperature2 + -3826.384 * Temperature3 + -243268.420 * Fuel_Price2 + -245343.569 * CPI2 + 38687.077 * Unemployment2 + 32259.480 * Holiday_Flag + 1033198.914

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.905957 |              126528 |         2.81869e+10 |           167889 |                         14.4068 |             -0.160197 |
+--------+----------------+---------------------+---------------------+----------------

LinearRegression(n_jobs=-1)

In [399]:
ignore = ['CPI2','Fuel_Price2','Temperature2','Unemployment2','Fuel_Price','Unemployment','Temperature','Holiday_Flag']
x_train2 = x_train.drop(ignore,axis=1)
x_test2 = x_test.drop(ignore,axis=1)
my_linear_regrassion(x_train2, y_train, x_test2, y_test, use_plot=False, report=True)

[회귀식]
Weekly_Sales = 11015.163 * CPI + 70063.874 * Date_value + 513367.006 * Store_value + -11963.356 * Temperature3 + 1035526.857

[회귀분석 성능평가]
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+
|        |   결정계수(R2) |   평균절대오차(MAE) |   평균제곱오차(MSE) |   평균오차(RMSE) |   평균 절대 백분오차 비율(MAPE) |   평균 비율 오차(MPE) |
|--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------|
| 데이터 |       0.903947 |              127384 |         2.87895e+10 |           169675 |                         14.4964 |             -0.189617 |
+--------+----------------+---------------------+---------------------+------------------+---------------------------------+-----------------------+

[선형회귀분석 결과보고]
+----+--------------+--------------+--------------------+------------+------------------+---------+------------+---------+
|    | 종속변수 

LinearRegression(n_jobs=-1)